In [53]:
import numpy as np 




def CREATE_MATRIX(VARIABLE,cons):    
    tab = np.zeros((cons+1, VARIABLE+cons+2))    
    return tab

#---------------------------------------------------------
def ROUNDOFFNEXT(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True
    
#---------------------------------------------------------
def ROUNDNEXT(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True
    
#---------------------------------------------------------
def FINDNEGATIVEVALUE(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

#---------------------------------------------------------
def FINDNEGATIVETABLE(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

#---------------------------------------------------------
def PIVOTLOCATE(table):
    total = []        
    r = FINDNEGATIVEVALUE(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]

#---------------------------------------------------------
def PIVOTLOCATION(table):
    if ROUNDNEXT(table):
        total = []
        n = FINDNEGATIVETABLE(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if b/i >0 and i**2>0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]
    
#---------------------------------------------------------
def pivot(row,col,table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')
        
#---------------------------------------------------------
def BOUNDCONVERT(eq):
    eq = eq.split(',')
    if 'U' in eq:
        g = eq.index('U')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

#---------------------------------------------------------

def BOUNDCONVERT_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

#---------------------------------------------------------

def GENERATEVARIABLEIABLE(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    VARIABLE = lc - lr -1
    v = []
    for i in range(VARIABLE):
        v.append('x'+str(i+1))
    return v

#---------------------------------------------------------
def CONSTRAINT_ADD(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i,:]:                       
            total += j**2
        if total == 0: 
            empty.append(total)
    if len(empty)>1:
        return True
    else:
        return False
    
#---------------------------------------------------------
def FUNCTIONCONSTRAINT(table,eq):
    if CONSTRAINT_ADD(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        VARIABLE = lc - lr -1      
        j = 0
        while j < lr:            
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:                
                row = row_check
                break
            j +=1
        eq = BOUNDCONVERT(eq)
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]
            i +=1        
        row[-1] = eq[-1]
        row[VARIABLE+j] = 1    
    else:
        print('Cannot add another FUNCTIONCONSTRAINTt.')
        
#---------------------------------------------------------
def OBJECTIVEECTADDFUNC(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0        
        for j in table[i,:]:
            total += j**2
        if total == 0:
            empty.append(total)    
    if len(empty)==1:
        return True
    else:
        return False
    
#---------------------------------------------------------
def OBJECTIVE(table,eq):
    if OBJECTIVEECTADDFUNC(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0        
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding FUNCTIONCONSTRAINTts before the OBJECTIVEective function can be added.')
        
#--------------------------------------------------------------
def OBJECTIVEECTMAXIMIZATION(table):
    while ROUNDOFFNEXT(table)==True:
        table = pivot(PIVOTLOCATE(table)[0],PIVOTLOCATE(table)[1],table)
    while ROUNDNEXT(table)==True:
        table = pivot(PIVOTLOCATION(table)[0],PIVOTLOCATION(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    VARIABLE = lc - lr -1
    i = 0
    val = {}
    for i in range(VARIABLE):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[GENERATEVARIABLEIABLE(table)[i]] = table[loc,-1]
        else:
            val[GENERATEVARIABLEIABLE(table)[i]] = 0
    val['Z'] = table[-1,-1]
    return val

#--------------------------------------------------------------
def OBJECTIVEECTMINIMIZATION(table):
    table = BOUNDCONVERT_min(table)
    while ROUNDOFFNEXT(table)==True:
        table = pivot(PIVOTLOCATE(table)[0],PIVOTLOCATE(table)[1],table)    
    while ROUNDNEXT(table)==True:
        table = pivot(PIVOTLOCATION(table)[0],PIVOTLOCATION(table)[1],table)       
    lc = len(table[0,:])
    lr = len(table[:,0])
    VARIABLE = lc - lr -1
    i = 0
    val = {}
    for i in range(VARIABLE):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]             
            val[GENERATEVARIABLEIABLE(table)[i]] = table[loc,-1]
        else:
            val[GENERATEVARIABLEIABLE(table)[i]] = 0 
            val['Z'] = table[-1,-1]*-1
    return val

#--------------------------------------------------------------

if __name__ == "__main__":
    
    #--------------------QUESTION 1 (1)----------------------------
    m = CREATE_MATRIX(4,3)
    FUNCTIONCONSTRAINT(m,'1, 2, 2, 4,L,40')
    FUNCTIONCONSTRAINT(m,'2, -1, 1, 2,L,8')
    FUNCTIONCONSTRAINT(m,'-4, 2, -1, 1,U,-10')
    OBJECTIVE(m,'2, 1, -3, 5')
    print("QUESTION 1 (1)",OBJECTIVEECTMAXIMIZATION(m)) 
    
    #--------------------QUESTION 1 (2)----------------------------
    m = CREATE_MATRIX(4,3)
    FUNCTIONCONSTRAINT(m,'1, 2, 2, 4,L,40')
    FUNCTIONCONSTRAINT(m,'2, -1, 1, 2,L,8')
    FUNCTIONCONSTRAINT(m,'-4, 2, -1, 1,U,-10')
    OBJECTIVE(m,'8, 6, 3, -2')
    print("QUESTION 1 (2)",OBJECTIVEECTMAXIMIZATION(m)) 
    
    #--------------------QUESTION 2----------------------------
    m = CREATE_MATRIX(4,3)
    FUNCTIONCONSTRAINT(m,'8, 3, 2, 2,L,10')
    FUNCTIONCONSTRAINT(m,'2, 5, 1, 4,L,5')
    FUNCTIONCONSTRAINT(m,'1, 2, 5, 1,L,6')
    OBJECTIVE(m,'3, 4, 1, 5')
    print("QUESTION 2",OBJECTIVEECTMAXIMIZATION(m))  
    
    #--------------------QUESTION 3----------------------------
    m = CREATE_MATRIX(3,3)
    FUNCTIONCONSTRAINT(m,'1, 1, 2,L,26')
    FUNCTIONCONSTRAINT(m,'1, 1, 2,L,26')
    FUNCTIONCONSTRAINT(m,'3, 2, 1,L,18')
    OBJECTIVE(m,'5, 3, 7')
    print("QUESTION 3",OBJECTIVEECTMAXIMIZATION(m)) 
    
    #--------------------QUESTION 4----------------------------
    m = CREATE_MATRIX(3,3)
    FUNCTIONCONSTRAINT(m,'3, -1, 2,L,7')
    FUNCTIONCONSTRAINT(m,'-2, 4, 0,L,12')
    FUNCTIONCONSTRAINT(m,'-4, 3, 8,L,10')
    OBJECTIVE(m,'1, -3, 2')
    print("QUESTION 4",OBJECTIVEECTMINIMIZATION(m))
    
    #--------------------QUESTION 5----------------------------
    m = CREATE_MATRIX(2,3)
    FUNCTIONCONSTRAINT(m,'10,5,L,250')
    FUNCTIONCONSTRAINT(m,'2,5,L,100')
    FUNCTIONCONSTRAINT(m,'2,3,L,90')
    OBJECTIVE(m,'-40,-100')
    print("QUESTION 5",OBJECTIVEECTMINIMIZATION(m))


QUESTION 1 (1) {'x1': 10.0, 'x2': 15.0, 'x3': 0, 'x4': 0, 'Z': 40.0}
QUESTION 1 (2) {'x1': 10.0, 'x2': 15.0, 'x3': 0, 'x4': 0, 'Z': 168.0}
QUESTION 2 {'x1': 1.0294117647058825, 'x2': 0.588235294117647, 'x3': 0, 'x4': 0, 'Z': 10.441176470588236}
QUESTION 3 {'x1': 6.0, 'x2': 0, 'x3': 0, 'Z': 37.0}
QUESTION 4 {'x1': 4.0, 'x2': 5.0, 'x3': 0, 'Z': -9.0}
QUESTION 5 {'x1': 25.0, 'x2': 0, 'Z': -1100.0}


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in double_scalars
